In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
np.random.seed(10)

In [ ]:
df = pd.read_csv('../input/quora-question-pairs/train.csv.zip')

In [ ]:
df

In [ ]:
df.describe()


In [ ]:
print('Total number of question pairs for training: {}'.format(len(df)))

print('Duplicate pairs: {}%'.format(round(df['is_duplicate'].mean()*100, 2)))
qids = pd.Series(df['qid1'].tolist() + df['qid2'].tolist())


print('Total number of questions in the training data: {}'.format(len(
    np.unique(qids))))


print('Number of questions that appear multiple times: {}'.format(np.sum(qids.value_counts() > 1)))


plt.figure(figsize=(12, 5))
plt.hist(qids.value_counts(), bins=50)
plt.yscale('log', nonposy='clip')
plt.title('Log-Histogram of question appearance counts')
plt.xlabel('Number of occurences of question')
plt.ylabel('Number of questions')
print()

In [ ]:

null = df.isnull().sum()
print(null)

df = df.dropna(subset = ['question1', 'question2'])

pd.value_counts(df['is_duplicate']).plot.bar()
plt.xlabel('Values')
plt.ylabel('Count')
plt.title('Class Distribution')

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(module_url)




In [ ]:
def UniversalEmbedding(x):
    return embed(tf.reshape(tf.cast(x, tf.string), [-1]))

DROPOUT = 0.1

In [ ]:

q1 = layers.Input(shape=(1,), dtype=tf.string)
embedding_q1 = layers.Lambda(UniversalEmbedding, output_shape=(512,))(q1)

In [ ]:


q2 = layers.Input(shape=(1,), dtype=tf.string)
embedding_q2 = layers.Lambda(UniversalEmbedding, output_shape=(512,))(q2)


merged = layers.concatenate([embedding_q1, embedding_q2])
merged = layers.Dense(200, activation='relu')(merged)
merged = layers.Dropout(DROPOUT)(merged)


merged = layers.BatchNormalization()(merged)
merged = layers.Dense(200, activation='relu')(merged)
merged = layers.Dropout(DROPOUT)(merged)

merged = layers.BatchNormalization()(merged)
merged = layers.Dense(200, activation='relu')(merged)
merged = layers.Dropout(DROPOUT)(merged)

merged = layers.BatchNormalization()(merged)
merged = layers.Dense(200, activation='relu')(merged)
merged = layers.Dropout(DROPOUT)(merged)


merged = layers.BatchNormalization()(merged)
pred = layers.Dense(2, activation='sigmoid')(merged)
model = Model(inputs=[q1,q2], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
q1= df.question1.tolist()
q2= df.question2.tolist()
labels= df.is_duplicate.tolist()

In [ ]:
from sklearn.model_selection import train_test_split
X1 = df['question1']
X2 = df['question2']
y = df['is_duplicate']

X1_train, X1_test,X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

train_q1 = X1_train.tolist()
train_q1 = np.array(train_q1, dtype=object)[:, np.newaxis]
train_q2 = X2_train.tolist()
train_q2 = np.array(train_q2, dtype=object)[:, np.newaxis]

train_labels = np.asarray(pd.get_dummies(y_train), dtype = np.int8)

test_q1 = X1_test.tolist()
test_q1 = np.array(test_q1, dtype=object)[:, np.newaxis]
test_q2 = X2_test.tolist()
test_q2 = np.array(test_q2, dtype=object)[:, np.newaxis]

test_labels = np.asarray(pd.get_dummies(y_test), dtype = np.int8)

In [ ]:
from keras.callbacks import ModelCheckpoint

# Creating the tensorflow session to train the model and save checkpoint after every epoch.
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session());



filepath="./model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=True, mode='auto', period=1)
callbacks_list = [checkpoint]

history = model.fit([train_q1, train_q2], 
        train_labels,
        validation_data=([test_q1, test_q2], test_labels),
        epochs=10,
        batch_size=512, callbacks=callbacks_list)
model.save('./final_model.h5')

In [ ]:
import numpy as np
import tensorflow as tf
from keras import backend as K
q1 = "What is one coin?	"
q2 =  "What's this coin?"
q1 = np.array([[q1],[q1]])
q2 = np.array([[q2],[q2]])


# Using the same tensorflow session for embedding the test string

tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session());
model.load_weights('./model.hdf5')  
# Predicting the similarity between the two input questions 
predicts = model.predict([q1, q2], verbose=0)
predict_logits = predicts.argmax(axis=1)
print("----FINAL RESULT----")
if(predict_logits[0] == 1):
    print("****Questions are Similar****")
else:
    print("****Questions are not Similar****")

In [ ]:
df